In [108]:
import pandas as pd
raw_df = pd.read_csv("data/match_map_stats.csv")

In [109]:
match_ids = raw_df.match_id.unique()
len(match_ids)

642

In [110]:
#The first 6 features of the dataframe can be easily set:
team_one_list = []
team_two_list = []
date_list = []
winner_list = []
stage_list = []

for m in match_ids:
    temp_df = raw_df[raw_df['match_id'] == m]
    team_one_list.append(temp_df.team_one_name.unique()[0])
    team_two_list.append(temp_df.team_two_name.unique()[0])
    date_list.append(temp_df.round_start_time.unique()[0])
    winner_list.append(temp_df.match_winner.unique()[0])
    stage_list.append(temp_df.stage.unique()[0])
    
#Let's zip this all up and make a dataframe!
zipped_data = list(zip(match_ids, team_one_list, team_two_list, date_list, stage_list, winner_list))
columns = ['id', 'team_one', 'team_two', 'date', 'stage', 'winner']    

df = pd.DataFrame(zipped_data, columns=columns)

#Make sure the date column is in the proper format
df['date']=pd.to_datetime(df['date'])

In [111]:
#Set the corona_virus_isolation feature.  This will be True if games are being played while in isolation because of the pandemic
df['corona_virus_isolation'] = False

#We will use a mask to properly set the feature:
mask = (df['date'] > '03/20/2020')
df['corona_virus_isolation'] = mask

In [112]:
#Initiate some features:
df['t1_wins_season'] = ""
df['t1_losses_season'] = ""
df['t2_wins_season'] = ""
df['t2_losses_season'] = ""
df['t1_matches_season'] = ""
df['t2_matches_season'] = ""

In [113]:
#Returned {team_one_wins, team_one_losses, team_one_total_matches, team_two_wins, team_two_losses, team_two_total_matches}
def get_win_loss_total(wl_df, wl_id):
    t1 = ((wl_df[wl_df['id'] == wl_id]).team_one).iloc[0]
    t2 = ((wl_df[wl_df['id'] == wl_id]).team_two).iloc[0]
    date = ((wl_df[wl_df['id'] == wl_id]).date).iloc[0]
    #print(f"Team 1: {t1} Team 2: {t2} Date: {date}")
    teams=[t1, t2]
    
    return_info=[]
    
    for team in teams:
        df_partial_season=wl_df[wl_df['date']<date]
        season_1 = df_partial_season[df_partial_season['team_one'] == team]
        season_2 = df_partial_season[df_partial_season['team_two'] == team]
        team_season = season_1.append(season_2)
        wins = len(team_season[team_season['winner'] == team])
        losses = len(team_season[team_season['winner'] != team])
        matches = wins+losses
        #print(f"{team} {len(season_1)} {len(season_2)} {len(team_season)} {wins} ")
        return_info.append(wins)
        return_info.append(losses)
        return_info.append(matches)
        
    return(return_info)


In [114]:
def find_team_match_results(master_df, min_date, max_date, type):
    df_reduced = master_df[master_df['date'] > min_date]
    df_reduced = df_reduced[df_reduced['date'] < max_date]
    
    #We can use this to make an ID list...
    id_list = df_reduced.id.unique()
    
    for i in id_list:
        win_loss_list = (get_win_loss_total(df_reduced, i))
        mask = master_df['id'] == i
        
        if type=='2020_season':
            master_df['t1_wins_season'][mask] = win_loss_list[0]
            master_df['t1_losses_season'][mask] = win_loss_list[1]
            master_df['t1_matches_season'][mask] = win_loss_list[2]
            master_df['t2_wins_season'][mask] = win_loss_list[3]
            master_df['t2_losses_season'][mask] = win_loss_list[4]
            master_df['t2_matches_season'][mask] = win_loss_list[5]    
    return(master_df)

In [115]:
results_2020 = find_team_match_results(df, '01/01/2020', '12/31/2020', '2020_season')


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWa

In [117]:
results_2020.to_csv('test2.csv')